<a href="https://colab.research.google.com/github/EduMoura321/Aeronautic-Aerospace-codes/blob/main/Rogers_aproximation_for_aerodynamics_force.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#2-aproximar polinomio de Roger
#========================================================
#aproxima com roger para pegar A0, A1 e Al
def ROGER(k,lags):
  #pega valores complexos Q
  Q = []
  for kr in k:
    C_k = (hankel2(1,kr))/(hankel2(1,kr)+1j*hankel2(0,kr))
    Q.append(C_k)
  Q = np.array(Q)
  #Faz polinomio de roger
  A = []
  for kl in k:
    p = 1j*kl
    row = [1,p]
    for b in lags:
      row.append(p/(p+b))
    A.append(row)
  A = np.array(A)

  A_stack = np.vstack([A.real, A.imag])
  Q_stack = np.hstack([Q.real, Q.imag])
  coefs, _, _, _ = np.linalg.lstsq(A_stack, Q_stack, rcond=None)
  return coefs
k = np.linspace(0,1,10)
lags = np.array([0.2,0.4,0.6,0.8])
APR = ROGER(k,lags)
A0 = APR[0]
A1 = APR[1]
Als = np.zeros(4)
Als = APR[2:]
#=========================================================
def mat_lag(lags):
  #matriz de betas e Ar
  betas = np.zeros((4,4))
  for i in range(len(lags)):
    betas[i,i] = lags[i]
  Br = betas@Als
  return betas, Br



In [ ]:
#3-Montar Matrizes

def Mats_estru(M, Ig, Xab, Kh, Ka ):
  M = np.array([[M, M*Xab*b],[M*Xab*b, Ig + M*(Xab*b)**2]]) #matriz de massa
  K = np.array([[Kh, 0],[0, Ka]]) #matriz de stiffness
  C = np.array([[0, 0],[0, 0]]) #matriz de damping
  Minv = np.linalg.inv(M) #matriz inversa de massa
  return K, C, Minv
#==============================================================================
#Sistema para solver
def System_aug(t,X):
  q = X[0:2]
  dq = X[2:4]
  z = X[4:7]
  Minv, K, C = Mats_estru(M, Ig, Xab, Kh, Ka )
  A, Br = mat_lag(lags)

  ddq = Minv @ ((A1-C)@dq + (A0-K)@q + F_aero(q)@z)
  dz = A@z + Br@np.hstack([q,dq])

  return np.concatenate([dq, ddq, dz])